In [134]:
import numpy as np
from numpy import vstack
import string
import matplotlib.pyplot as plt
import plotly as py
import plotly.graph_objs as go
print(py.__version__)
from collections import defaultdict
from Hilbert_Curve_Generator_3D import HilbertCurveGenerator_3D

4.7.1


This version uses a hardcoded list of rules and dimensions to determine the coordinates. See version 2 for code that uses original function curve_index2 to dynamically generate rules list based on dimensions and order of alphabet

In [140]:
def hilbert_index(seq):
    '''
    Converts input sequence into matrix index 
    for space filling curve in 4 quadrants ABCD
    
    Parameters:
    ----------
    seq: String
        Input string to be converted 
        
    Returns
    -------
    index: list
        List of x,y matrix indices 
    '''
    rules = {'A': np.array([0, 0, 0]),
             'B': np.array([0, 1, 0]),
             'C': np.array([0, 1, 1]),
             'D': np.array([0, 0, 1]),
             'E': np.array([1, 0, 1]),
             'F': np.array([1, 1, 1]),
             'G': np.array([1, 1, 0]),
             'H': np.array([1, 0, 0])
            }
    dim = 2
    index = np.array([])
    
    for order, val in enumerate(seq[::-1].upper()):
        if index.size == 0:
            index = np.zeros_like(rules[val])
        index += (dim ** order) * rules[val] 
    
    return index.tolist()

In [144]:
def normalize_value(val, level):
    result = (val*2+1)/(2**(level+1))
    return result

In [123]:
def normalize_coordinates(coords, i):
    text = ["x= ", "y= "]
    new_coords = ""
    for j, val in enumerate(coords):
        new_coords += text[j]
        new_val = normalize_value(val, i)
        new_coords += str(new_val)
        new_coords += " "
    return new_coords

In [148]:
level = 2
def draw_curve():

    # Create figure
    fig = go.Figure()

    hc_gen = HilbertCurveGenerator_3D()
    hc_seq = hc_gen.generate_curve(plot_depth=level)
    
    coordinates = []
    for i, seq in enumerate(hc_seq):
        coordinates.append(hilbert_index(seq))

    #x_vals = [x for [x,y,z] in coordinates]
    #y_vals = [y for [x,y,z] in coordinates]
    #z_vals = [z for [x,y,z] in coordinates]
    x_vals = [normalize_value(x, level) for [x,y,z] in coordinates]
    y_vals = [normalize_value(y, level) for [x,y,z] in coordinates]
    z_vals = [normalize_value(z, level) for [x,y,z] in coordinates]

    for point in coordinates:
        fig.add_trace(
                go.Scatter3d(
                    x=x_vals,
                    y=y_vals,
                    z=z_vals,
                    name='Hilbert Curve level: ' + str(level),
                    mode='lines+markers',
                    marker=dict(
                        size=7,
                        color=x_vals,
                        colorscale='Viridis'),
                    line=dict(
                        color='#677193',
                        width=5),
                    visible=False))

    for i, point in enumerate(coordinates):
            
            #point
            hc_point = point
            x_val = hc_point[0]
            x_val = normalize_value(x_val, level)

            y_val = hc_point[1]
            y_val = normalize_value(y_val, level)
            
            z_val = hc_point[2]
            z_val = normalize_value(z_val, level)

            fig.add_trace(go.Scatter3d(x = [x_val],
                                 y = [y_val],
                                 z = [z_val],
                                 name='Point corresponding to sequence ' + str(hc_seq[i]),
                                 mode='markers',
                                 visible=False,
                                 marker=dict(
                                       size=10,
                                       color='#e377c2')))
    # Create and add slider
    steps = []

    for i, st in enumerate(fig.data[::2]):
            step = dict(
                method="update",
                args=[{"visible": [False] * len(fig.data[::2])},
                      {"title": "Coordinates"}],
                label='Level {}'.format(i))
            step["args"][0]["visible"][i] = True  # Toggle i'th trace to "visible"
            steps.append(step)

    # Make last trace visible
    fig.data[0].visible = True
    fig.data[len(coordinates)].visible = True

    sliders = [dict(
        active=0,
        currentvalue={"prefix": ""},
        steps=steps
    )]

    fig.layout.plot_bgcolor = 'white'
    
    camera = dict(
    eye=dict(x=1, y=-1, z=.65)
    )

    fig.update_layout(
        scene_camera=camera,
        title="3D curve",
        sliders=sliders,
        height=700,
        scene = dict(xaxis = dict(range=[0,1],),
                     yaxis = dict(range=[0,1],),
                     zaxis = dict(range=[0,1],))
    )

    fig.show()


In [149]:
draw_curve()